In [1]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity

In [31]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader.dataset)
    accuracy = 100. * correct / total
    return train_loss, accuracy

In [32]:
import torch

def evaluate(device, model, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [146]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.from_numpy(np.array([KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([64]) for a in activations], dtype="float32")).squeeze(2)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(
                    extract_activations_per_sample(
                            extract_activations_layers(layers), 
                            mask=False
                        )
                ), 
                bandwidth=0.2
            )

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, 64)
        self.l2 = LinW(in_features=64, out_features=64, depth=0)
        self.l3 = LinW(in_features=64, out_features=64, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(64, 10)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input* wd(prev).to('cuda:0'), self.weight, self.bias)

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


LinW layers:

Depth 0: LinW(in_features=64, out_features=64, bias=True)
Depth 1: LinW(in_features=64, out_features=64, bias=True)
Epoch 1/10, Training Loss: 1.6687, Training Accuracy: 39.01%, Test accuracy: 49.06%
Epoch 2/10, Training Loss: 1.3445, Training Accuracy: 51.48%, Test accuracy: 53.64%
Epoch 3/10, Training Loss: 1.2649, Training Accuracy: 54.51%, Test accuracy: 56.10%
Epoch 4/10, Training Loss: 1.2258, Training Accuracy: 55.48%, Test accuracy: 57.01%
Epoch 5/10, Training Loss: 1.1836, Training Accuracy: 56.70%, Test accuracy: 56.09%
Epoch 6/10, Training Loss: 1.1564, Training Accuracy: 57.52%, Test accuracy: 57.70%
Epoch 7/10, Training Loss: 1.1321, Training Accuracy: 58.13%, Test accuracy: 57.77%
Epoch 8/10, Training Loss: 1.1127, Training Accuracy: 59.04%, Test accuracy: 59.86%
Epoch 9/10, Training Loss: 1.0984, Training Accuracy: 59.62%, Test accuracy: 59.93%
Epoch 10/10, Training Loss: 1.0898, Training Accuracy: 59.99%, Test accuracy: 61.62%


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.from_numpy(np.array([KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([64]) for a in activations], dtype="float32")).squeeze(2)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(
                    extract_activations_per_sample(
                            extract_activations_layers(layers), 
                            mask=False
                        )
                ), 
                bandwidth=0.2
            )

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, 64)
        self.l2 = LinW(in_features=64, out_features=64, depth=0)
        self.l3 = LinW(in_features=64, out_features=64, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(64, 10)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(wd(prev).to('cuda:0'), self.weight, self.bias)

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


LinW layers:

Depth 0: LinW(in_features=64, out_features=64, bias=True)
Depth 1: LinW(in_features=64, out_features=64, bias=True)
Epoch 1/10, Training Loss: 2.3016, Training Accuracy: 11.37%, Test accuracy: 11.24%
Epoch 2/10, Training Loss: 2.2997, Training Accuracy: 11.81%, Test accuracy: 11.93%
Epoch 3/10, Training Loss: 2.2986, Training Accuracy: 12.07%, Test accuracy: 12.17%
Epoch 4/10, Training Loss: 2.2990, Training Accuracy: 11.91%, Test accuracy: 11.67%
Epoch 5/10, Training Loss: 2.2993, Training Accuracy: 11.83%, Test accuracy: 12.25%
Epoch 6/10, Training Loss: 2.2988, Training Accuracy: 12.08%, Test accuracy: 12.16%


KeyboardInterrupt: 

In [34]:
""" In this experiment we introduce a mutation magnitude over the 
activations in the intermediate layers. Since the activations mutate
we still introduce an additional dynamic weight that play an important
role in the training process to update the weights of the network.

"""


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity

import random


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.from_numpy(np.array([KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([64]) for a in activations], dtype="float32")).squeeze(2)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(
                    extract_activations_per_sample(
                            extract_activations_layers(layers), 
                            mask=False
                        )
                ), 
                bandwidth=0.2
            )

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, 64)
        self.l2 = LinW(in_features=64, out_features=64, depth=0)
        self.l3 = LinW(in_features=64, out_features=64, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(64, 10)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input*random.uniform(0.99,1.09), self.weight, self.bias)

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


LinW layers:

Depth 0: LinW(in_features=64, out_features=64, bias=True)
Depth 1: LinW(in_features=64, out_features=64, bias=True)
Epoch 1/10, Training Loss: 0.3766, Training Accuracy: 88.75%, Test accuracy: 93.53%
Epoch 2/10, Training Loss: 0.1644, Training Accuracy: 95.06%, Test accuracy: 95.93%
Epoch 3/10, Training Loss: 0.1139, Training Accuracy: 96.59%, Test accuracy: 96.23%
Epoch 4/10, Training Loss: 0.0916, Training Accuracy: 97.22%, Test accuracy: 96.74%
Epoch 5/10, Training Loss: 0.0743, Training Accuracy: 97.71%, Test accuracy: 97.07%
Epoch 6/10, Training Loss: 0.0606, Training Accuracy: 98.08%, Test accuracy: 96.70%
Epoch 7/10, Training Loss: 0.0522, Training Accuracy: 98.39%, Test accuracy: 96.80%
Epoch 8/10, Training Loss: 0.0446, Training Accuracy: 98.58%, Test accuracy: 97.38%
Epoch 9/10, Training Loss: 0.0388, Training Accuracy: 98.82%, Test accuracy: 97.44%
Epoch 10/10, Training Loss: 0.0337, Training Accuracy: 98.93%, Test accuracy: 97.44%


In [38]:
""" Here insted of multiply I sum the sampled activations
of the previous layers on the inputs.



"""


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.from_numpy(np.array([KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([64]) for a in activations], dtype="float32")).squeeze(2)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(
                    extract_activations_per_sample(
                            extract_activations_layers(layers), 
                            mask=False
                        )
                ), 
                bandwidth=0.2
            )

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, 64)
        self.l2 = LinW(in_features=64, out_features=64, depth=0)
        self.l3 = LinW(in_features=64, out_features=64, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(64, 10)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        return F.linear(input+wd(prev).to('cuda:0'), self.weight, self.bias)

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


LinW layers:

Depth 0: LinW(in_features=64, out_features=64, bias=True)
Depth 1: LinW(in_features=64, out_features=64, bias=True)
Epoch 1/10, Training Loss: 0.7291, Training Accuracy: 75.94%, Test accuracy: 82.01%
Epoch 2/10, Training Loss: 0.5565, Training Accuracy: 82.27%, Test accuracy: 83.00%
Epoch 3/10, Training Loss: 0.5347, Training Accuracy: 82.81%, Test accuracy: 83.62%
Epoch 4/10, Training Loss: 0.5326, Training Accuracy: 83.13%, Test accuracy: 83.76%
Epoch 5/10, Training Loss: 0.5273, Training Accuracy: 83.41%, Test accuracy: 82.78%
Epoch 6/10, Training Loss: 0.5334, Training Accuracy: 83.11%, Test accuracy: 82.91%
Epoch 7/10, Training Loss: 0.5336, Training Accuracy: 83.21%, Test accuracy: 84.01%
Epoch 8/10, Training Loss: 0.5358, Training Accuracy: 83.08%, Test accuracy: 83.01%
Epoch 9/10, Training Loss: 0.5353, Training Accuracy: 83.09%, Test accuracy: 82.71%
Epoch 10/10, Training Loss: 0.5376, Training Accuracy: 83.33%, Test accuracy: 83.13%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.from_numpy(np.array([KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([64]) for a in activations], dtype="float32")).squeeze(2)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(
                    extract_activations_per_sample(
                            extract_activations_layers(layers), 
                            mask=False
                        )
                ), 
                bandwidth=0.2
            )

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, 64)
        self.l2 = LinW(in_features=64, out_features=64, depth=0)
        self.l3 = LinW(in_features=64, out_features=64, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(64, 10)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        print(wd(prev).shape)
        return F.linear(input, self.weight* wd(prev).to('cuda:0'), self.bias)

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


In [37]:
layers = [torch.randn(16,256) for i in range(5)]

layers = np.array([np.array(i.flatten()) for i in layers]).flatten()
mask = layers > 0
layers = layers[mask]

res = layers[np.random.randint(0, high = len(layers), size = 256)].reshape(-1, 1)

array([[0.5981148 ],
       [0.9634854 ],
       [1.1180352 ],
       [0.25245157],
       [2.484147  ],
       [1.161348  ],
       [0.87380683],
       [0.44336656],
       [0.9320248 ],
       [1.1274574 ],
       [0.12749925],
       [1.7562745 ],
       [0.7457105 ],
       [0.08640575],
       [1.2357011 ],
       [1.8209292 ],
       [1.2359884 ],
       [0.30162323],
       [0.12449323],
       [0.05077058],
       [0.42775792],
       [0.5930919 ],
       [0.6066395 ],
       [0.26709583],
       [1.0321921 ],
       [0.07010241],
       [0.12023831],
       [0.1930366 ],
       [1.2029365 ],
       [0.21384852],
       [0.667683  ],
       [0.22509019],
       [0.97664076],
       [0.33046725],
       [1.3761361 ],
       [0.6719827 ],
       [0.3749767 ],
       [0.51912063],
       [2.6440165 ],
       [0.3438414 ],
       [0.44473463],
       [0.35270986],
       [0.8643006 ],
       [0.46024805],
       [0.7238416 ],
       [1.5828304 ],
       [1.4176172 ],
       [0.280

In [18]:

def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.from_numpy(np.array([KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([64]) for a in activations], dtype="float32")).squeeze(2)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(
                    extract_activations_per_sample(
                            extract_activations_layers(layers), 
                            mask=False
                        )
                ), 
                bandwidth=0.2
            )


import random

layers = [torch.randn(64,5) for i in range(5)]

layers = extract_activations_per_sample(extract_activations_layers(layers))
layers = [l[l < 0] for l in layers]


[random.choice(l) for l in layers]

# res = layers[np.random.randint(0, high = layers.shape[0]-1, size = 256)].reshape(-1, 1)
# return torch.from_numpy(np.array(res, dtype="float32"))


# wd(layers).shape

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, 256)
        self.l4 = nn.Linear(256, 10)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.gelu(self.l2(x))
        x = self.gelu(self.l3(x))
        x = self.l4(x)
        return x
    

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLP().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10, Training Loss: 1.8164, Training Accuracy: 33.93%, Test accuracy: 39.72%
Epoch 2/10, Training Loss: 1.6439, Training Accuracy: 40.74%, Test accuracy: 41.99%
Epoch 3/10, Training Loss: 1.5454, Training Accuracy: 44.20%, Test accuracy: 42.73%
Epoch 4/10, Training Loss: 1.4934, Training Accuracy: 45.90%, Test accuracy: 44.51%
Epoch 5/10, Training Loss: 1.4409, Training Accuracy: 48.04%, Test accuracy: 49.23%
Epoch 6/10, Training Loss: 1.4050, Training Accuracy: 49.34%, Test accuracy: 47.79%
Epoch 7/10, Training Loss: 1.3646, Training Accuracy: 50.74%, Test accuracy: 48.76%
Epoch 8/10, Training Loss: 1.3404, Training Accuracy: 51.69%, Test accuracy: 48.74%
Epoch 9/10, Training Loss: 1.3010, Training Accuracy: 53.03%, Test accuracy: 49.75%
Epoch 10/10, Training Loss: 1.2674, Training Accuracy: 54.31%, Test accuracy: 49.79%


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5

# prev = [torch.randn(12,64) for i in range(N_layer)]

# wd(prev).shape


class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, 64)
        self.l2 = LinW(in_features=64, out_features=64, depth=0)
        self.l3 = LinW(in_features=64, out_features=64, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(64, 10)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(wd(prev).to(device), self.weight, self.bias)

EPOCHS = 10
BATCH_SIZE = 64

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
LinW layers:

Depth 0: LinW(in_features=64, out_features=64, bias=True)
Depth 1: LinW(in_features=64, out_features=64, bias=True)
Epoch 1/10, Training Loss: 2.1590, Training Accuracy: 22.46%, Test accuracy: 24.77%
Epoch 2/10, Training Loss: 2.0821, Training Accuracy: 25.27%, Test accuracy: 25.72%
Epoch 3/10, Training Loss: 2.0594, Training Accuracy: 25.38%, Test accuracy: 26.05%
Epoch 4/10, Training Loss: 2.0429, Training Accuracy: 26.14%, Test accuracy: 27.14%
Epoch 5/10, Training Loss: 2.0360, Training Accuracy: 26.53%, Test accuracy: 26.75%
Epoch 6/10, Training Loss: 2.0265, Training Accuracy: 26.52%, Test accuracy: 26.78%


KeyboardInterrupt: 

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5

# prev = [torch.randn(12,64) for i in range(N_layer)]

# wd(prev).shape

n_neurons = 8

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, n_neurons)
        self.l2 = LinW(in_features=n_neurons, out_features=n_neurons, depth=0)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(wd(prev).to(device), self.weight, self.bias)

EPOCHS = 8
BATCH_SIZE = 120

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
LinW layers:

Depth 0: LinW(in_features=8, out_features=8, bias=True)
Depth 1: LinW(in_features=8, out_features=8, bias=True)
Epoch 1/8, Training Loss: 1.9131, Training Accuracy: 29.59%, Test accuracy: 33.05%
Epoch 2/8, Training Loss: 1.8118, Training Accuracy: 34.05%, Test accuracy: 33.90%
Epoch 3/8, Training Loss: 1.8016, Training Accuracy: 34.52%, Test accuracy: 34.20%
Epoch 4/8, Training Loss: 1.8011, Training Accuracy: 34.51%, Test accuracy: 35.09%
Epoch 5/8, Training Loss: 1.7988, Training Accuracy: 34.48%, Test accuracy: 34.99%
Epoch 6/8, Training Loss: 1.7970, Training Accuracy: 34.71%, Test accuracy: 34.54%
Epoch 7/8, Training Loss: 1.7936, Training Accuracy: 34.67%, Test accuracy: 34.90%
Epoch 8/8, Training Loss: 1.7837, Training Accuracy: 35.40%, Test accuracy: 35.95%


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5

# prev = [torch.randn(12,64) for i in range(N_layer)]

# wd(prev).shape

n_neurons = 64

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, n_neurons)
        self.l2 = LinW(in_features=n_neurons, out_features=n_neurons, depth=0)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons, depth=1, layers=[self.l2])
        self.l4 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()
        self.layers = [self.l2, self.l3]

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features, depth, layers=[]):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)
        self.depth = depth
        self.layers = layers[:self.depth] if len(layers)>0 else layers

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(wd(prev).to(device), self.weight, self.bias)

EPOCHS = 8
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
LinW layers:

Depth 0: LinW(in_features=64, out_features=64, bias=True)
Depth 1: LinW(in_features=64, out_features=64, bias=True)
Epoch 1/8, Training Loss: 0.7540, Training Accuracy: 75.49%, Test accuracy: 82.97%
Epoch 2/8, Training Loss: 0.5118, Training Accuracy: 83.48%, Test accuracy: 85.69%
Epoch 3/8, Training Loss: 0.4610, Training Accuracy: 85.07%, Test accuracy: 85.93%


KeyboardInterrupt: 

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5

# prev = [torch.randn(12,64) for i in range(N_layer)]

# wd(prev).shape

n_neurons = 120

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(784, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l4 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(wd(prev).to(device), self.weight, self.bias)

EPOCHS = 8
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=784, out_features=120, bias=True)
  (l2): Linear(in_features=120, out_features=120, bias=True)
  (l3): LinW(in_features=120, out_features=120, bias=True)
  (l4): Linear(in_features=120, out_features=10, bias=True)
  (layer_norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/8, Training Loss: 0.4732, Training Accuracy: 85.15%, Test accuracy: 90.38%
Epoch 2/8, Training Loss: 0.2742, Training Accuracy: 91.38%, Test accuracy: 92.41%
Epoch 3/8, Training Loss: 0.2385, Training Accuracy: 92.53%, Test accuracy: 92.82%
Epoch 4/8, Training Loss: 0.2137, Training Accuracy: 93.28%, Test accuracy: 93.04%
Epoch 5/8, Training Loss: 0.1997, Training Accuracy: 93.62%, Test accuracy: 93.70%
Epoch 6/8, Training Loss: 0.1803, Training Accuracy: 94.31%, Test accuracy: 93.83%
Epoch 7/8, Training Loss: 0.1697, Training Accuracy: 94.67%, Test accuracy: 93.88%
Epoch 

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5

# prev = [torch.randn(12,64) for i in range(N_layer)]

# wd(prev).shape

n_neurons = 256

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l4 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()
        # self.softmax = nn.Softmax()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.l4(x)
        # x = self.softmax(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input * wd(prev).to(device), self.weight, self.bias)

EPOCHS = 40
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=256, bias=True)
  (l2): LinW(in_features=256, out_features=256, bias=True)
  (l3): LinW(in_features=256, out_features=256, bias=True)
  (l4): Linear(in_features=256, out_features=10, bias=True)
  (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/40, Training Loss: 1.9619, Training Accuracy: 27.99%, Test accuracy: 34.12%
Epoch 2/40, Training Loss: 1.7523, Training Accuracy: 36.41%, Test accuracy: 39.03%
Epoch 3/40, Training Loss: 1.6934, Training Accuracy: 39.15%, Test accuracy: 40.33%
Epoch 4/40, Training Loss: 1.6638, Training Accuracy: 40.22%, Test accuracy: 40.92%
Epoch 5/40, Training Loss: 1.6374, Training Accuracy: 40.95%, Test accuracy: 41.85%
Epoch 6/40, Training Loss: 1.6202, Training Accuracy: 41.79%, Test accuracy: 41.31%
Epoch 7/

In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 120

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = nn.Linear(n_neurons, n_neurons)
        self.l4 = nn.Linear(n_neurons, n_neurons)
        # self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l5 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l6 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l4(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l5(x, repr))
        x = self.layer_norm(x)
        x = self.l6(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(wd(prev).to(device), self.weight, self.bias)

EPOCHS = 50
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=120, bias=True)
  (l2): Linear(in_features=120, out_features=120, bias=True)
  (l3): LinW(in_features=120, out_features=120, bias=True)
  (l4): Linear(in_features=120, out_features=120, bias=True)
  (l5): LinW(in_features=120, out_features=120, bias=True)
  (l6): Linear(in_features=120, out_features=10, bias=True)
  (layer_norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/50, Training Loss: 2.1340, Training Accuracy: 21.17%, Test accuracy: 25.31%
Epoch 2/50, Training Loss: 2.0068, Training Accuracy: 27.19%, Test accuracy: 28.33%
Epoch 3/50, Training Loss: 1.9659, Training Accuracy: 28.54%, Test accuracy: 29.18%
Epoch 4/50, Training Loss: 1.9438, Training Accuracy: 29.51%, Test accuracy: 29.77%
Epoch 5/50, Training Loss: 1.9298, Training Accuracy: 

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 120

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = nn.Linear(n_neurons, n_neurons)
        self.l4 = nn.Linear(n_neurons, n_neurons)
        # self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l5 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l6 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l4(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l5(x, repr))
        x = self.layer_norm(x)
        x = self.l6(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input + wd(prev).to(device), self.weight, self.bias)

EPOCHS = 50
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=120, bias=True)
  (l2): Linear(in_features=120, out_features=120, bias=True)
  (l3): LinW(in_features=120, out_features=120, bias=True)
  (l4): Linear(in_features=120, out_features=120, bias=True)
  (l5): LinW(in_features=120, out_features=120, bias=True)
  (l6): Linear(in_features=120, out_features=10, bias=True)
  (layer_norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/50, Training Loss: 2.0278, Training Accuracy: 24.53%, Test accuracy: 33.27%
Epoch 2/50, Training Loss: 1.8030, Training Accuracy: 34.68%, Test accuracy: 36.84%
Epoch 3/50, Training Loss: 1.7368, Training Accuracy: 37.09%, Test accuracy: 39.38%
Epoch 4/50, Training Loss: 1.6867, Training Accuracy: 39.54%, Test accuracy: 38.17%
Epoch 5/50, Training Loss: 1.6643, Training Accuracy: 

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 120

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = nn.Linear(n_neurons, n_neurons)
        self.l4 = nn.Linear(n_neurons, n_neurons)
        # self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l5 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l6 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l4(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l5(x, repr))
        x = self.layer_norm(x)
        x = self.l6(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input * wd(prev).to(device), self.weight, self.bias)

EPOCHS = 50
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=120, bias=True)
  (l2): Linear(in_features=120, out_features=120, bias=True)
  (l3): LinW(in_features=120, out_features=120, bias=True)
  (l4): Linear(in_features=120, out_features=120, bias=True)
  (l5): LinW(in_features=120, out_features=120, bias=True)
  (l6): Linear(in_features=120, out_features=10, bias=True)
  (layer_norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/50, Training Loss: 2.1603, Training Accuracy: 17.04%, Test accuracy: 24.12%
Epoch 2/50, Training Loss: 1.9425, Training Accuracy: 28.18%, Test accuracy: 29.56%
Epoch 3/50, Training Loss: 1.8612, Training Accuracy: 32.12%, Test accuracy: 33.63%
Epoch 4/50, Training Loss: 1.8200, Training Accuracy: 34.00%, Test accuracy: 35.00%
Epoch 5/50, Training Loss: 1.7953, Training Accuracy: 

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 120

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = nn.Linear(n_neurons, n_neurons)
        self.l4 = nn.Linear(n_neurons, n_neurons)
        # self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        # self.l5 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l6 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l4(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        # x = self.gelu(self.l5(x, repr))
        # x = self.layer_norm(x)
        x = self.l6(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input * wd(prev).to(device), self.weight, self.bias)

EPOCHS = 50
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=120, bias=True)
  (l2): Linear(in_features=120, out_features=120, bias=True)
  (l3): Linear(in_features=120, out_features=120, bias=True)
  (l4): Linear(in_features=120, out_features=120, bias=True)
  (l6): Linear(in_features=120, out_features=10, bias=True)
  (layer_norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/50, Training Loss: 2.0233, Training Accuracy: 25.62%, Test accuracy: 33.15%
Epoch 2/50, Training Loss: 1.7594, Training Accuracy: 36.43%, Test accuracy: 38.83%
Epoch 3/50, Training Loss: 1.6658, Training Accuracy: 40.35%, Test accuracy: 43.26%
Epoch 4/50, Training Loss: 1.6232, Training Accuracy: 41.66%, Test accuracy: 39.48%
Epoch 5/50, Training Loss: 1.5897, Training Accuracy: 42.67%, Test accuracy: 42.88%
Epoch 6/50, Training Loss: 

In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 32

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = nn.Linear(n_neurons, n_neurons)
        self.l4 = nn.Linear(n_neurons, n_neurons)
        # self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l5 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l6 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l4(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l5(x, repr))
        x = self.layer_norm(x)
        x = self.l6(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input * wd(prev).to(device), self.weight, self.bias)

EPOCHS = 50
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=32, bias=True)
  (l2): Linear(in_features=32, out_features=32, bias=True)
  (l3): LinW(in_features=32, out_features=32, bias=True)
  (l4): Linear(in_features=32, out_features=32, bias=True)
  (l5): LinW(in_features=32, out_features=32, bias=True)
  (l6): Linear(in_features=32, out_features=10, bias=True)
  (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/50, Training Loss: 2.2789, Training Accuracy: 13.29%, Test accuracy: 18.11%
Epoch 2/50, Training Loss: 2.0588, Training Accuracy: 22.38%, Test accuracy: 26.33%
Epoch 3/50, Training Loss: 1.9576, Training Accuracy: 27.87%, Test accuracy: 30.61%
Epoch 4/50, Training Loss: 1.9080, Training Accuracy: 30.32%, Test accuracy: 30.30%
Epoch 5/50, Training Loss: 1.8719, Training Accuracy: 31.44%, Tes

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 256

class MLPWD(nn.Module):
    def __init__(self):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.l2 = nn.Linear(n_neurons, n_neurons)
        self.l3 = nn.Linear(n_neurons, n_neurons)
        self.l4 = nn.Linear(n_neurons, n_neurons)
        # self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l5 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l6 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l3(x, repr))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l4(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l5(x, repr))
        x = self.layer_norm(x)
        x = self.l6(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input * wd(prev).to(device), self.weight, self.bias)

EPOCHS = 50
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): LinW(in_features=256, out_features=256, bias=True)
  (l4): Linear(in_features=256, out_features=256, bias=True)
  (l5): LinW(in_features=256, out_features=256, bias=True)
  (l6): Linear(in_features=256, out_features=10, bias=True)
  (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (gelu): GELU(approximate='none')
)
Epoch 1/50, Training Loss: 2.1420, Training Accuracy: 17.96%, Test accuracy: 21.17%
Epoch 2/50, Training Loss: 1.9382, Training Accuracy: 27.77%, Test accuracy: 29.20%
Epoch 3/50, Training Loss: 1.8583, Training Accuracy: 31.81%, Test accuracy: 32.32%
Epoch 4/50, Training Loss: 1.8294, Training Accuracy: 32.86%, Test accuracy: 34.87%
Epoch 5/50, Training Loss: 1.7890, Training Accuracy: 

KeyboardInterrupt: 

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 64

class MLPWD(nn.Module):
    def __init__(self, n_head=2):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.t_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=n_neurons, nhead=n_head), num_layers=2)
        self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.t_encoder(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        x = self.layer_norm(x)
        x = self.l3(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input * wd(prev).to(device), self.weight, self.bias)

EPOCHS = 20
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=64, bias=True)
  (t_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamical

In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
from sklearn.neighbors import KernelDensity


def extract_activations_layers(layers):
    """ Extract for each layer the activations

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (layer_activation, batch_size, number_of_activations)
    """

    return np.array([np.array([np.array(h) for h in l]) for l in layers])

def extract_activations_per_sample(layers, mask = False):
    """ Extract for each sample the activations 
    for each layer and store them in a list.

    Args:
        layers (np.array): shape (layer_activation, batch_size, number_of_neurons)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    if mask == True:
        # mask the activations to remove zeros
        mask = layers != 0
        layers = [[np.array(h[m]) for h, m in zip(l,sm)] 
                for l, sm in zip(layers, mask)]
        
    return np.array([layers[:,i,:].flatten().reshape(-1, 1) for i in range(layers.shape[1])])


def get_sampled_activations(activations, bandwidth = 0.2):
    """ Sample the activations using KDE

    Args:
        activations (np.array): shape (batch_size, number_of_activations)

    Returns:
        np.array: shape (batch_size, number_of_activations)
    """

    return torch.stack([
                    torch.tensor(KernelDensity(kernel="gaussian", bandwidth=bandwidth).fit(a).sample([1]), dtype=torch.float32).squeeze(0) 
                    for a in activations], dim=0)

def wd(layers: list()):
    """ Compute the weight decay for each layer

    Args:
        layers (list): list of layers

    Returns:
        torch.tensor: weight decay

    """
    return get_sampled_activations(
                list(torch.from_numpy(extract_activations_layers(layers)).permute(1,0,2).numpy()), 
                bandwidth=0.2
            )
    
    
# N_layer = 5
# prev = [torch.randn(12,64) for i in range(N_layer)]
# wd(prev).shape

n_neurons = 64

class MLPWD(nn.Module):
    def __init__(self, n_head=2):
        super(MLPWD, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3072, n_neurons)
        self.t_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=n_neurons, nhead=n_head), num_layers=2)
        self.l2 = LinW(in_features=n_neurons, out_features=n_neurons)
        self.l3 = nn.Linear(n_neurons, 10)
        self.layer_norm = nn.LayerNorm(n_neurons)
        self.gelu = nn.GELU()

    def forward(self, x):
        repr = []
        x = self.flatten(x)
        x = self.gelu(self.l1(x))
        x = self.layer_norm(x)
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.t_encoder(x))
        repr.append(x.detach().cpu().numpy())
        x = self.gelu(self.l2(x, repr))
        x = self.layer_norm(x)
        x = self.l3(x)
        return x
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)
    

class LinW(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinW, self).__init__(in_features=in_features, out_features=out_features)

    def forward(self, input, prev=[]):
        # weight_decay = wd(prev)
        # weight = self.weight * weight_decay.to('cuda:0')
        # return F.linear(input, weight, self.bias)
        # print(wd(prev).shape)
        return F.linear(input + wd(prev).to(device), self.weight, self.bias)

EPOCHS = 20
BATCH_SIZE = 256

# check if mps are available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

print("Device:", device)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8, T_mult=1, eta_min=5e-3)
# StepLR(optimizer, step_size=10, gamma=0.5)

print(model)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()

Device: mps
Files already downloaded and verified
Files already downloaded and verified
MLPWD(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=3072, out_features=64, bias=True)
  (t_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamical